# Volumetrics: HCIP calculation

We'll implement the volumetric equation:

$$ V = A \times T \times G \times \phi \times N\!\!:\!\!G \times S_\mathrm{O} \times \frac{1}{B_\mathrm{O}} $$

## Gross rock volume 

$$ \mathrm{GRV} = A \times T $$

## Geometric factor

Now we need to compensate for the prospect not being a flat slab of rock &mdash; using the geometric factor. 

We will implement the equations implied by this diagram:

<img src="http://subsurfwiki.org/images/6/66/Geometric_correction_factor.png", width=600>

Let's turn this one into a function too. It's a little trickier:

Apply the geometric factor to the gross rock volume:

## Multiple prospects

In [1]:
thicknesses = [10, 25, 15, 5, 100]
heights = [75, 100, 20, 100, 200]

## HC pore volume

$$ \mathrm{HCPV} = \mathrm{N\!:\!G} \times \phi \times S_\mathrm{O} $$

We need:

- net:gross &mdash; the ratio of reservoir-quality rock thickness to the total thickness of the interval.
- porosity
- $S_\mathrm{O}$ &mdash; the oil saturation, or proportion of oil to total pore fluid.

### EXERCISE

Turn this into a function by rearranging the following lines of code:

    """A function to compute the hydrocarbon pore volume."""
    return hcpv
    hcpv = netg * por * s_o
    def calculate_hcpv(netg, por, s_o):


In [ ]:
# Put your code here:




After you define the function and run that cell, this should work:

In [ ]:
calculate_hcpv(0.5, 0.24, 0.8)

You should get: `0.096`.

## Formation volume factor

Oil shrinks when we produce it, especially if it has high GOR. The FVF, or $B_O$, is the ratio of a reservoir barrel to a stock-tank barrel (25 deg C and 1 atm). Typically the FVF is between 1 (heavy oil) and 1.7 (high GOR).

In [ ]:
fvf = 1.1

We could define something to remember the FVF for different types of oil:

### EXERCISE

For gas, $B_\mathrm{G}$ is $0.35 Z T / P$, where $Z$ is the correction factor, or gas compressibility factor. $T$ should be in kelvin and $P$ in kPa. $Z$ is usually between 0.8 and 1.2, but it can be as low as 0.3 and as high as 2.0.

Can you write a function to calculate $B_\mathrm{G}$?

In [ ]:
def calculate_Bg(     ):  # Add the arguments.
    """Write a docstring."""
    
    
    return         # Don't forget to return something!

## Put it all together

Now we have the components of the volumetric equation:

[For more on conversion to bbl, BOE, etc.](https://en.wikipedia.org/wiki/Barrel_of_oil_equivalent)

### EXERCISE

- Can you write a function to compute the volume (i.e. the HCIP), given all the inputs?
- Try to use the functions for calculating GRV and HCPV that you have already written.

As a reminder, here's the equation:

$$ V = A \times T \times G \times \phi \times N\!\!:\!\!G \times S_\mathrm{O} \times \frac{1}{B_\mathrm{O}} $$

In [ ]:
# Put your code here.


    

When you've defined the function, this should work:

In [ ]:
calculate_hcip(area, thick, g, por, netg, s_o, fvf)

You should get `4189090909`.

## Monte Carlo simulation

We can easily draw randomly from distributions of properties:

- Normal: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.normal.html
- Uniform: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.uniform.html
- Lognormal: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.lognormal.html

The histogram looks a bit ragged, but this is probably because of the relatively small number of samples.

### EXERCISE

1. How does the histogram look if you take 1000 or 10,000 samples instead of 100?
1. Make distributions for some of the other properties, like thickness and FVF.
1. Maybe our functions should check that we don't get unreasonable values, like negative numbers, or decimal fractions over 1.0 Try to implement this if you have time.

----

## Full MC calculation

Remember how when we changed lists of multiple values into our functions, they didn't work, but if we used arrays they did? Well, our distributions are arrays, so we might just be able to pass them straight to our function:

In [ ]:
calculate_hcip(area, thick, g, por, netg, s_o, fvf)

In [ ]:
hcip = calculate_hcip(area, thick, g, por, netg, s_o, fvf)

plt.figure(figsize=(20,4))
plt.bar(np.arange(100), sorted(hcip))
plt.show()

In [ ]:
p = 50

cols = 100 * ['gray']
cols[p] = 'red'

plt.figure(figsize=(20,4))
plt.bar(np.arange(100), sorted(hcip), color=cols)
plt.text(p, hcip[p]*1.3, f'{hcip[50]:.2e} $\mathrm{{Sm}}^3$', ha='center', color='red', size=20)
plt.show()

### Lognormal distributions

We might prefer a lognormal distribution for some parameters, e.g. area and porosity.

This is a little trickier, and involves using `scipy`. The good news is that this gives us access to 97 other continuous distributions, as well as multivariate distributions and other useful things.

In [ ]:
import scipy.stats

dist = scipy.stats.lognorm(s=0.2, scale=0.15)

This has instantiated a continuous distribution, from which we can now sample random variables:

In [ ]:
samples = dist.rvs(size=10000)

These have a lognormal distribution.

In [ ]:
_ = plt.hist(samples, bins=40)

# Reading data from a file

Let's try reading data from a CSV.

In [ ]:
ls ../data

In [ ]:
!head -5 ../data/HC_volumes_random_input.csv

In [ ]:
with open('../data/HC_volumes_random_input.csv') as f:
    for line in f:
        data = line.split(',')
        print(data)

In [ ]:
import csv
with open('../data/HC_volumes_random_input.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row)

In [ ]:
row

In [ ]:
row['phi']

## Using `pandas`

In [ ]:
import pandas as pd

df = pd.read_csv('../data/HC_volumes_random_input.csv')

In [ ]:
df.head()

In [ ]:
df[['Name', 'Thick [m]']]

In [ ]:
calculate_grv(df['Thick [m]'], df['Area [km2]'])

We could also compute a distribution for each row in the dataframe:

In [ ]:
df.columns

In [ ]:
def wrapper(row):
    _, name, thick, area, g, netg, por, s_o, fvf, grv = row
    area *= 1000000
    return calculate_hcip(area, thick, g, por, netg, s_o, fvf)

In [ ]:
df.apply(wrapper, axis=1)

In [ ]:
df['HCIP'] = df.apply(wrapper, axis=1)

In [ ]:
df.head()

<hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2018</p>
</div>